In [40]:
"""
给出了使用朴素贝叶斯进行垃圾邮件分类的整体框架
但将部分内容进行了挖空，你需要将其填充完成，以保证程序的正常运行以及输出正确结果
注意：
    代码的其他部分不需要更改，只需填充上所给出的代码块，否则会影响评判。
    需要填空的地方格式如下：
    ###### start ######

    ######  end  ######
"""

'\n给出了使用朴素贝叶斯进行垃圾邮件分类的整体框架\n但将部分内容进行了挖空，你需要将其填充完成，以保证程序的正常运行以及输出正确结果\n注意：\n    代码的其他部分不需要更改，只需填充上所给出的代码块，否则会影响评判。\n    需要填空的地方格式如下：\n    ###### start ######\n\n    ######  end  ######\n'

同学们做之前先花一点时间读读关于此次任务的讲解吧！

![info1](./info1.png)

In [41]:
# -*- coding: UTF-8 -*-
import numpy as np
import re
import random

In [42]:
"""
函数说明:将切分的实验样本词条整理成不重复的词条列表，也就是词汇表
Parameters:
    dataSet - 整理的样本数据集
Returns:
    vocabSet - 返回不重复的词条列表，也就是词汇表
"""
def createVocabList(dataSet):
    vocabSet = set([])  # 创建一个空的不重复列表
    for document in dataSet:
        vocabSet = vocabSet | set(document)  # 取并集
    return list(vocabSet)
 
 


In [43]:
"""
函数说明:根据vocabList词汇表，将inputSet向量化，向量的每个元素为1或0
Parameters:
    vocabList - createVocabList返回的列表
    inputSet - 切分的词条列表
Returns:
    returnVec - 文档向量,词集模型
"""
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)               #创建一个其中所含元素都为0的向量
    for word in inputSet:                          #遍历每个词条
        if word in vocabList:                      #如果词条存在于词汇表中，则置1
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word: %s is not in my Vocabulary!" % word)
    return returnVec        #返回文档向量
 

In [44]:
"""
函数说明:朴素贝叶斯分类器训练函数
Parameters:
    trainMatrix - 训练文档矩阵，即setOfWords2Vec返回的returnVec构成的矩阵
    trainCategory - 训练类别标签向量，即loadDataSet返回的classVec
Returns:
    p0Vect - 正常邮件类的条件概率数组
    p1Vect - 垃圾邮件类的条件概率数组
    pAbusive - 文档属于垃圾邮件类的概率
"""
def trainNB0(trainMatrix, trainCategory):
    ###### start ######
    numTrainDocs = len(trainMatrix)  # 计算训练的文档数目，即样本数
    ######  end  ######
   
    numWords = len(trainMatrix[0])  # 计算每篇文档的词条数
    
    ###### start ######
    pAbusive = sum(trainCategory)/numTrainDocs  # 文档属于垃圾邮件类的概率
    ######  end  ######
    
    p0Num = np.ones(numWords)
    p1Num = np.ones(numWords)  # 创建numpy.ones数组,词条出现数初始化为1,拉普拉斯平滑
    p0Denom = 2.0
    p1Denom = 2.0  # 分母初始化为2 ,拉普拉斯平滑
    for i in range(numTrainDocs):
        
        ###### start ######
        if trainCategory[i] == 1:  # 统计属于垃圾邮件类的条件概率所需的数据
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:  # 统计属于正常邮件类的条件概率所需的数据
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
        ######  end  ######
    ###### start ######
    p1Vect = np.log(p1Num/p1Denom) # 即P(w0|1),P(w1|1),P(w2|1)···
    p0Vect = np.log(p0Num/p0Denom) # 即P(w0|0),P(w1|0),P(w2|0)···
    ######  end  ######
      
    return p0Vect, p1Vect, pAbusive  # 返回属于正常邮件类的条件概率数组，属于侮辱垃圾邮件类的条件概率数组，文档属于垃圾邮件类的概率
 


下面代码块所填内容参考以下公式
![info2](./info2.png)

In [45]:
"""
函数说明:朴素贝叶斯分类器分类函数
Parameters:
	vec2Classify - 待分类的词条数组
	p0Vec - 正常邮件类的条件概率数组
	p1Vec - 垃圾邮件类的条件概率数组
	pClass1 - 文档属于垃圾邮件的概率
Returns:
	0 - 属于正常邮件类
	1 - 属于垃圾邮件类
"""
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    
    ###### start ######
        p1 = sum(vec2Classify*p1Vec) + pClass1
        p0 = sum(vec2Classify*p0Vec) + 1 - pClass1
        # print(p1)
        # print(p0)
        if p1 > p0:
            return 1
        else:
            return 0
    ######  end  ######


In [46]:
"""
函数说明:接收一个大字符串并将其解析为字符串列表
"""
def textParse(bigString):  # 将字符串转换为字符列表

    listOfTokens = re.split(r'\W+', bigString)  # 将特殊符号作为切分标志进行字符串切分，即非字母、非数字

    return [tok.lower() for tok in listOfTokens if len(tok) > 2]  # 除了单个字母，例如大写的I，其它单词变成小写
 


In [47]:
"""
函数说明:测试朴素贝叶斯分类器，使用朴素贝叶斯进行交叉验证
"""
def spamTest():
    docList = []
    classList = []
    fullText = []
    for i in range(1, 26):  # 遍历25个txt文件

        wordList = textParse(open('email/spam/%d.txt' % i, 'r', encoding='ISO-8859-1').read())  # 读取每个垃圾邮件，并字符串转换成字符串列表
        docList.append(wordList)
        fullText.append(wordList)
        classList.append(1)  # 标记垃圾邮件，1表示垃圾文件
        wordList = textParse(open('email/ham/%d.txt' % i, 'r', encoding='ISO-8859-1').read())  # 读取每个非垃圾邮件，并字符串转换成字符串列表
        docList.append(wordList)
        fullText.append(wordList)
        classList.append(0)  # 标记正常邮件，0表示正常文件
#         print(docList)
#         print(classList)
    vocabList = createVocabList(docList)  # 创建词汇表，不重复
#     print(vocabList)
    trainingSet = list(range(50))
    testSet = []  # 创建存储训练集的索引值的列表和测试集的索引值的列表
    random.seed(0)
    for i in range(10):  # 从50个邮件中，随机挑选出40个作为训练集,10个做测试集
        randIndex = int(random.uniform(0, len(trainingSet)))  # 随机选取索索引值
        testSet.append(trainingSet[randIndex])  # 添加测试集的索引值
        del (trainingSet[randIndex])  # 在训练集列表中删除添加到测试集的索引值
    trainMat = []
    trainClasses = []  # 创建训练集矩阵和训练集类别标签系向量
    for docIndex in trainingSet:  # 遍历训练集
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))  # 将生成的词集模型添加到训练矩阵中
        trainClasses.append(classList[docIndex])  # 将类别添加到训练集类别标签系向量中
    p0V, p1V, pSpam = trainNB0(np.array(trainMat), np.array(trainClasses))  # 训练朴素贝叶斯模型
    errorCount = 0  # 错误分类计数
    for docIndex in testSet:  # 遍历测试集
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])  # 测试集的词集模型
        if classifyNB(np.array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:  # 如果分类错误
            errorCount += 1  # 错误计数加1
            print("分类错误的测试集：", docList[docIndex])
    print('错误率：%.2f%%' % (float(errorCount) / len(testSet) * 100))
 
 


In [48]:
# 如果看到了这里，说明同学们已经将上述代码读完了，那么调用哪个函数将程序跑起来呢？
###### start ######
spamTest()
######  end  ######

错误率：0.00%
